In [2]:
"""

import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Get the API key from the .env file
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

"""

Using Mistral 7b from Hugging Face (sharded ver.)

In [40]:

origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

### Embedding with vector db

In [6]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## Splitters

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 20,
    length_function = len,
)

### Loading content

In [25]:
from langchain.document_loaders import PyPDFDirectoryLoader


loader = PyPDFDirectoryLoader("data/")
docs = loader.load_and_split(text_splitter=text_splitter)
print(len(docs))


384


"FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other"

"The from_documents() method is called on the FAISS class, passing in two arguments: docs and embeddings. The docs parameter is a set of documents that have been previously loaded and split, and the embeddings parameter is a pre-trained embedding model that maps the documents to high-dimensional vectors."

In [29]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(docs, embeddings)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}